In [2]:
import torch
import numpy as np

<b>做一个线性回归模型</b>

In [4]:
x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)
print(x_train)
x_train = x_train.reshape(-1,1)
print(x_train)
x_train.shape

[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10.]
[[ 0.]
 [ 1.]
 [ 2.]
 [ 3.]
 [ 4.]
 [ 5.]
 [ 6.]
 [ 7.]
 [ 8.]
 [ 9.]
 [10.]]


(11, 1)

In [5]:
y_values = [2*i +1 for i in x_values]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1, 1)
y_train.shape

(11, 1)

In [6]:
import torch.nn as nn

In [7]:
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out

In [8]:
input_dim = 1
output_dim = 1
model = LinearRegressionModel(input_dim, output_dim)

In [10]:
# 指定好参数和损失函数
epoches = 1000
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

In [11]:
# 训练模型
for epoch in range(epoches):
    epoch += 1
    # 注意转成tensor
    inputs = torch.from_numpy(x_train)
    labels = torch.from_numpy(y_train)

    # 每一次迭代梯度需要清零
    optimizer.zero_grad()

    # 前向传播
    outputs = model(inputs)

    # 计算损失
    loss = criterion(outputs, labels)

    # 反向传播
    loss.backward()

    # 更新权重参数
    optimizer.step()

    # 打印
    if epoch % 50 ==0:
        print('epoch{}, loss {}'.format(epoch, loss.item()))

epoch50, loss 0.024334000423550606
epoch100, loss 0.01387922465801239
epoch150, loss 0.007916188798844814
epoch200, loss 0.0045150709338486195
epoch250, loss 0.002575250808149576
epoch300, loss 0.0014688298106193542
epoch350, loss 0.000837746832985431
epoch400, loss 0.0004778284637723118
epoch450, loss 0.0002725330414250493
epoch500, loss 0.00015544405323453248
epoch550, loss 8.866175630828366e-05
epoch600, loss 5.0569164159242064e-05
epoch650, loss 2.884113200707361e-05
epoch700, loss 1.6449959730380215e-05
epoch750, loss 9.381511517858598e-06
epoch800, loss 5.351242180040572e-06
epoch850, loss 3.052105057577137e-06
epoch900, loss 1.7408248140782234e-06
epoch950, loss 9.936321703207796e-07
epoch1000, loss 5.666946663041017e-07


In [19]:
# 测试模型预测结果
predicted = model(torch.from_numpy(x_train).requires_grad_()).data.numpy()
predicted

array([[ 0.99859977],
       [ 2.9988015 ],
       [ 4.9990034 ],
       [ 6.9992046 ],
       [ 8.999407  ],
       [10.999609  ],
       [12.99981   ],
       [15.000011  ],
       [17.000214  ],
       [19.000416  ],
       [21.000618  ]], dtype=float32)

In [20]:
# 模型的保存和读取
torch.save(model.state_dict(), 'model.pkl')

In [21]:
model.load_state_dict(torch.load('model.pkl'))

<All keys matched successfully>

<b>使用GPU进行训练</b>

In [22]:
# 只需要将数据和模型传入到CUDA里面就可以了
import torch
import torch.nn as nn
import numpy as np


class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    
    def forward(self, x):
        out = self.linear(x)
        return out

In [23]:
input_dim = 1
output_dim = 1
model = LinearRegressionModel(input_dim, output_dim)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 将模型传入到cuda中
model.to(device)

# 将数据传入到cuda中
criterion = nn.MSELoss()

learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [24]:
epoches = 1000
for epoch in range(epoches):
    epoch += 1
    inputs = torch.from_numpy(x_train).to(device)
    labels = torch.from_numpy(y_train).to(device)

    optimizer.zero_grad()

    outputs = model(inputs)

    loss = criterion(outputs, labels)

    loss.backward()

    optimizer.step()

    if epoch % 50 == 0:
        print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 50, loss 0.06610669195652008
epoch 100, loss 0.03770475089550018
epoch 150, loss 0.02150539867579937
epoch 200, loss 0.012265871278941631
epoch 250, loss 0.006995995994657278
epoch 300, loss 0.003990269731730223
epoch 350, loss 0.002275898354128003
epoch 400, loss 0.001298086135648191
epoch 450, loss 0.0007403779309242964
epoch 500, loss 0.0004222851130180061
epoch 550, loss 0.0002408587752142921
epoch 600, loss 0.00013737376139033586
epoch 650, loss 7.835499854991212e-05
epoch 700, loss 4.4691478251479566e-05
epoch 750, loss 2.548962766013574e-05
epoch 800, loss 1.4538189134327695e-05
epoch 850, loss 8.292179700220004e-06
epoch 900, loss 4.729485226562247e-06
epoch 950, loss 2.6980078473570757e-06
epoch 1000, loss 1.5391015040222555e-06
